# 2026 MCM C题 - 数据预处理

---

| 项目 | 说明 |
|------|------|
| **数据来源** | COMAP 官方竞赛数据 |
| **目的** | 数据清洗、转换、特征工程 |
| **注意** | 每步操作需确认后执行 |

---

## 1. 环境配置

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from cycler import cycler

warnings.filterwarnings('ignore')

# 竞赛风格配置
COLORS = {
    'primary': '#3C5488',
    'secondary': '#E64B35',
    'tertiary': '#00A087',
}
SCIENTIFIC_COLORS = ['#E64B35', '#4DBBD5', '#00A087', '#3C5488', '#F39B7F', '#8491B4']

plt.rcParams.update({
    'font.sans-serif': ['SimHei', 'Arial'],
    'axes.unicode_minus': False,
    'axes.prop_cycle': cycler('color', SCIENTIFIC_COLORS),
    'figure.figsize': (10, 6),
    'figure.dpi': 100,
})

print('环境配置完成')

环境配置完成


## 2. 加载原始数据

In [2]:
# 加载原始数据
DATA_PATH = 'data/2026_MCM_Problem_C_Data.csv'
df_raw = pd.read_csv(DATA_PATH)

print(f'原始数据: {df_raw.shape[0]} 行 x {df_raw.shape[1]} 列')
print(f'赛季范围: 第{df_raw.season.min()}季 - 第{df_raw.season.max()}季')

原始数据: 421 行 x 53 列
赛季范围: 第1季 - 第34季


In [3]:
# 创建工作副本（保留原始数据不变）
df = df_raw.copy()
print('已创建工作副本 df，原始数据 df_raw 保持不变')

已创建工作副本 df，原始数据 df_raw 保持不变


## 3. 数据预处理步骤

### 3.1 将 N/A 替换为 0

In [4]:
# 统计替换前的 N/A 数量
na_count_before = (df == 'N/A').sum().sum()
print(f'替换前 N/A 数量: {na_count_before}')

# 将所有 N/A 替换为 0
df = df.replace('N/A', 0)

# 将评分列转换为数值类型
score_cols = [c for c in df.columns if 'week' in c and 'judge' in c]
for col in score_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

na_count_after = (df == 'N/A').sum().sum()
print(f'替换后 N/A 数量: {na_count_after}')
print('N/A 替换完成')

替换前 N/A 数量: 0
替换后 N/A 数量: 0
N/A 替换完成


### 3.2 添加每周总分列

In [5]:
# 为每周添加总分列
for week in range(1, 12):
    week_cols = [f'week{week}_judge{j}_score' for j in range(1, 5)]
    existing_cols = [c for c in week_cols if c in df.columns]
    if existing_cols:
        df[f'week{week}_total'] = df[existing_cols].sum(axis=1)

# 显示新增的总分列
total_cols = [c for c in df.columns if '_total' in c]
print(f'新增 {len(total_cols)} 列周总分:')
print(total_cols)

# 显示前5行的总分
print('\n前5行周总分预览:')
print(df[['celebrity_name'] + total_cols].head())

新增 11 列周总分:
['week1_total', 'week2_total', 'week3_total', 'week4_total', 'week5_total', 'week6_total', 'week7_total', 'week8_total', 'week9_total', 'week10_total', 'week11_total']

前5行周总分预览:
      celebrity_name  week1_total  week2_total  week3_total  week4_total  \
0      John O'Hurley         20.0         26.0         24.0         21.0   
1       Kelly Monaco         13.0         17.0         21.0         26.0   
2  Evander Holyfield         18.0         14.0         13.0          0.0   
3      Rachel Hunter         20.0         24.0         26.0         25.0   
4      Joey McIntyre         20.0         21.0         22.0         20.0   

   week5_total  week6_total  week7_total  week8_total  week9_total  \
0         27.0         27.0          0.0          0.0          0.0   
1         23.5         27.5          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4 

### 3.3 添加每周淘汰标记列

In [6]:
# 添加每周淘汰标记 (0=未淘汰, 1=淘汰)
# 逻辑: 如果本周总分>0 且 下周总分=0，则本周被淘汰

for week in range(1, 11):  # 第1-10周可能被淘汰
    current_col = f'week{week}_total'
    next_col = f'week{week+1}_total'
    
    if current_col in df.columns and next_col in df.columns:
        # 本周有分数，下周没分数 = 本周被淘汰
        df[f'week{week}_eliminated'] = ((df[current_col] > 0) & (df[next_col] == 0)).astype(int)

# 第11周特殊处理：进入第11周但未获得前3名的选手
if 'week11_total' in df.columns:
    # 第11周有分数但名次>3 = 第11周被淘汰
    df['week11_eliminated'] = ((df['week11_total'] > 0) & (df['placement'] > 3)).astype(int)

# 显示新增的淘汰标记列
elim_cols = [c for c in df.columns if '_eliminated' in c]
print(f'新增 {len(elim_cols)} 列淘汰标记:')
print(elim_cols)

# 统计每周淘汰人数
print('\n每周淘汰人数统计:')
for col in elim_cols:
    week_num = col.split('_')[0]
    elim_count = df[col].sum()
    print(f'  {week_num}: {elim_count} 人')

新增 11 列淘汰标记:
['week1_eliminated', 'week2_eliminated', 'week3_eliminated', 'week4_eliminated', 'week5_eliminated', 'week6_eliminated', 'week7_eliminated', 'week8_eliminated', 'week9_eliminated', 'week10_eliminated', 'week11_eliminated']

每周淘汰人数统计:
  week1: 17 人
  week2: 44 人
  week3: 34 人
  week4: 38 人
  week5: 27 人
  week6: 33 人
  week7: 37 人
  week8: 39 人
  week9: 43 人
  week10: 72 人
  week11: 10 人


In [7]:
# 验证淘汰标记
print('淘汰标记验证（前10行）:')
verify_cols = ['celebrity_name', 'results', 'placement'] + elim_cols
print(df[verify_cols].head(10).to_string())

淘汰标记验证（前10行）:
      celebrity_name            results  placement  week1_eliminated  week2_eliminated  week3_eliminated  week4_eliminated  week5_eliminated  week6_eliminated  week7_eliminated  week8_eliminated  week9_eliminated  week10_eliminated  week11_eliminated
0      John O'Hurley          2nd Place          2                 0                 0                 0                 0                 0                 1                 0                 0                 0                  0                  0
1       Kelly Monaco          1st Place          1                 0                 0                 0                 0                 0                 1                 0                 0                 0                  0                  0
2  Evander Holyfield  Eliminated Week 3          5                 0                 0                 1                 0                 0                 0                 0                 0                 0                  0  

In [8]:
# 数据预处理后的列结构
print('=' * 50)
print('预处理后数据结构')
print('=' * 50)
print(f'总行数: {len(df)}')
print(f'总列数: {len(df.columns)}')
print(f'\n新增列:')
print(f'  - 周总分列: {len(total_cols)} 列')
print(f'  - 淘汰标记列: {len(elim_cols)} 列')
print('=' * 50)

预处理后数据结构
总行数: 421
总列数: 75

新增列:
  - 周总分列: 11 列
  - 淘汰标记列: 11 列
